## NOTE
1. To run this notebook you need to have colab pro or plus as the free version will run out of memory.
2. An alternative you can run this notebook locally if u have 16+ gb RAM and good GPU
3. [More illustrated example](https://colab.research.google.com/github/nengelmann/Fuyu-8B---Exploration/blob/main/Fuyu_8B_Exploration.ipynb#scrollTo=SRz1AjrWPZsk)

In [ ]:
!pip install transformers accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.2 MB/s eta 0:00:00


In [ ]:
from transformers import FuyuForCausalLM, FuyuProcessor, FuyuImageProcessor, AutoTokenizer
from PIL import Image
import torch

model_name = "ybelkada/fuyu-8b-sharded"
tokenizer = AutoTokenizer.from_pretrained(model_name)

image_processor = FuyuImageProcessor()
processor = FuyuProcessor(
    image_processor = image_processor,
    tokenizer = tokenizer
)

model = FuyuForCausalLM.from_pretrained(
    model_name,
    device_map = "cuda",
    torch_dtype = torch.float16,
)

In [ ]:
text_prompt = "Generate a coco-style caption."
image_path = "bus.png"
image_pil = Image.open(image_path)
text_prompt += "\n"

In [ ]:
model_inputs = processor(
    text=text_prompt,
    images=[image_pil],
    device="cuda"
)
for k, v in model_inputs.items():
    model_inputs[k] = v.to("cuda")

In [ ]:
generation_output = model.generate(**model_inputs, max_new_tokens=7)
generation_text = processor.batch_decode(generation_output[:, -7:], skip_special_tokens=True)
print(generation_text[0])